# Exploratory Data Analysis: Online Retail Dataset  
*Author: Pragun Sapotra*  
*Date: August 2025*

This notebook explores the "Online Retail" dataset to understand data structure, types, and basic statistics.  
It sets the foundation for subsequent data cleaning and analysis tasks.


### Importing libraries

In [3]:
import os
import pandas as pd

### Define file path to Excel dataset

In [4]:
base_dir = os.getcwd()
file_path = os.path.join(base_dir, "../Data/global_indicators_raw.xlsx")

### Retrieving Sheet Names from the Excel File

In [5]:
xlsx = pd.ExcelFile(file_path)
print(xlsx.sheet_names)


['Online Retail']


### Load Data from 'Online Retail' Sheet  
Read the 'Online Retail' sheet into a DataFrame for analysis.


In [6]:
raw_df = pd.read_excel(file_path, sheet_name='Online Retail')
df = raw_df.copy()

### Exploring Data

In [7]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


### Initial Observations:
- The Description and CustomerID columns have some missing values, which might need cleaning.
- CustomerID is stored as a float, which seems unusual because IDs are usually strings or integers. I think this might be because of those missing values (NaNs) causing pandas to use float by default.
- InvoiceNo has many repeated values, which could mean multiple items per invoice or duplicate records — I need to check that further.


### Handling Duplicates
In this section, I check for duplicate rows, inspect them, and filter specific cases for further investigation.


In [9]:
df.duplicated().sum()

np.int64(5268)

The dataset contains 5268 fully duplicated rows.


In [10]:
df[df.duplicated()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom
555,536412,22327,ROUND SNACK BOXES SET OF 4 SKULLS,1,2010-12-01 11:49:00,2.95,17920.0,United Kingdom
...,...,...,...,...,...,...,...,...
541675,581538,22068,BLACK PIRATE TREASURE CHEST,1,2011-12-09 11:34:00,0.39,14446.0,United Kingdom
541689,581538,23318,BOX OF 6 MINI VINTAGE CRACKERS,1,2011-12-09 11:34:00,2.49,14446.0,United Kingdom
541692,581538,22992,REVOLVER WOODEN RULER,1,2011-12-09 11:34:00,1.95,14446.0,United Kingdom
541699,581538,22694,WICKER STAR,1,2011-12-09 11:34:00,2.10,14446.0,United Kingdom


Viewing the duplicate rows reveals that they have identical values across all columns.


Example: Filtering duplicates with StockCode = 22111 and InvoiceNo = 536409.


In [16]:
df[ (df["StockCode"] == 22111) & (df["InvoiceNo"] == 536409	) ]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom


### Removing Duplicate Rows


In [15]:
df.drop_duplicates(inplace=True)

**Removed all exact duplicate rows** across the entire dataset using `drop_duplicates()` to ensure each record is unique.  
Kept the **first occurrence** of each unique row and dropped the rest.  

- **Total rows before:** 541,909  
- **Total rows after:** 536,641  


### Missing Values Analysis
Checking for null values in each column to identify data gaps before cleaning.


In [18]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135037
Country             0
dtype: int64

## Missing Values Overview
- `Description`: 1,454 missing entries.
- `CustomerID`: 135,037 missing entries — likely guest or anonymous customers.
- All other columns have no missing values.


In [29]:
df[df['Description'].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1970,536545,22139,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1971,536546,22139,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1972,536547,22139,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1987,536549,22139,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535322,581199,22139,NaN,-2,2011-12-07 18:26:00,0.0,NaN,United Kingdom
535326,581203,22139,NaN,15,2011-12-07 18:31:00,0.0,NaN,United Kingdom
535332,581209,22139,NaN,6,2011-12-07 18:35:00,0.0,NaN,United Kingdom
536981,581234,22139,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom


**Observations on Missing `Description` Values:**  
- All rows retrieved with `Above command` have a **`UnitPrice` of 0.0**.  
- All of them also have a **missing (`NaN`) `CustomerID`**.  
- At least one row has a **negative `Quantity`**, which could indicate a return.  
- This suggests these records might represent **invalid or placeholder transactions**, but this needs full validation before deciding how to handle them.  


In [33]:
df[(df["Description"].isnull()) & (df["UnitPrice"]== 0.0) & (df["CustomerID"].isnull())]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1970,536545,22139,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1971,536546,22139,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1972,536547,22139,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1987,536549,22139,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535322,581199,22139,NaN,-2,2011-12-07 18:26:00,0.0,NaN,United Kingdom
535326,581203,22139,NaN,15,2011-12-07 18:31:00,0.0,NaN,United Kingdom
535332,581209,22139,NaN,6,2011-12-07 18:35:00,0.0,NaN,United Kingdom
536981,581234,22139,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom


**Observations on Missing `Description` Values:**  
- All 1454 rows with missing `Description` also have:  
  - **`UnitPrice` of 0.0**  
  - **Missing (`NaN`) `CustomerID`**  
  - The same **`StockCode` = 22139**  
- These rows appear to be non-usable for analysis and may need to be removed.  

In [35]:
df.loc[df["StockCode"] == 22139, "Description"].dropna().unique()


array(['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN',
       'CREAM CUPID HEARTS COAT HANGER', ..., 'lost',
       'CREAM HANGING HEART T-LIGHT HOLDER',
       'PAPER CRAFT , LITTLE BIRDIE'], shape=(4223,), dtype=object)

**Investigation of Missing `Description` Values:**
- Checked if missing `Description` rows could be filled using their `StockCode`.
- Found that `StockCode 22139` (which appears in all such rows) maps to **multiple different descriptions** in the dataset.
- Since there is no reliable 1-to-1 mapping, filling these values could introduce incorrect data.
- Decision: **Drop these rows** instead of imputing a value.


In [37]:
df.drop(df [(   df["Description"].isnull())
             & (df["UnitPrice"]== 0.0) 
             & (df["CustomerID"].isnull())
            ].index,
        inplace=True)

**Removed Invalid Placeholder Transactions**  
Dropped rows where:  
- `Description` is missing (`NaN`)  
- `UnitPrice` equals `0.0`  
- `CustomerID` is missing (`NaN`)  

These records appear to be non-usable placeholder entries.  

- **Total rows before:** 536,641  
- **Total rows after:** 535,187  
- **Rows removed:** 1,454  


In [ ]:
## df[df["CustomerID"].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,22139,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,22139,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,22139,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,22139,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,22139,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541537,581498,22139,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541538,581498,22139,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
541539,581498,22139,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom
